In [ ]:
import warnings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._mocks import profiles

In [ ]:
field_centre = [0, 0]
side_lengths = [20, 24]
penumbra_width = 2
rotation = 20

field = profiles.create_rectangular_field_function(
    field_centre, side_lengths, penumbra_width, rotation)

In [ ]:
def show_field_with_profiles(field):
    x = np.arange(-20, 20.1, 0.1)
    xx, yy = np.meshgrid(x, x)
    
    plt.figure()
    plt.pcolormesh(xx, yy, field(xx, yy))
    plt.axis('equal')
    
    plt.figure()
    plt.plot(x, field(x, 0))
    
    plt.figure()
    plt.plot(x, field(0, x))
    
    
show_field_with_profiles(field)

In [ ]:
t = np.linspace(0, 2*np.pi)
plt.plot(np.sin(t), np.cos(t))
plt.axis('equal')

In [ ]:
x = np.linspace(-1, 1, 100)
y = np.sqrt(1 - x**2)

plt.plot(x, y)
plt.axis('equal')

In [ ]:
bb_diameter = 8

x = np.linspace(0, bb_diameter + 4, 1000)
y = np.sqrt(bb_diameter**2 - x**2) / bb_diameter

y[np.isnan(y)] = 0

plt.plot(x, y)
plt.plot(x + penumbra_width/3, y)

In [ ]:
import sympy

In [ ]:
# From https://codereview.stackexchange.com/a/202614
def convolve(f, g, t, lower_limit=-sympy.oo, upper_limit=sympy.oo):
    tau = sympy.Symbol('__very_unlikely_name__', real=True)
    return sympy.integrate(f.subs(t, tau) * g.subs(t, t - tau), 
                     (tau, lower_limit, upper_limit))

In [ ]:
x = sympy.Symbol('x')
r = sympy.Symbol('r')
circle = sympy.sqrt(r - x**2)

y = sympy.Symbol('y')
sig = sympy.Symbol('\sigma')
mu = sympy.Symbol('\mu')

gauss = 1/(sig * sympy.sqrt(2*sympy.pi)) * sympy.exp(-1/2 * ((y - mu) / sig)**2)


gauss

In [ ]:
circle

In [ ]:
# convolve(circle, gauss, 0.5, 0, 1)

In [ ]:
import scipy.ndimage

In [ ]:
def create_bb_attenuation_func(diameter, penumbra, max_attenuation):
    dx = diameter / 100
    radius = diameter / 2
    image_half_width = penumbra * 2 + radius
    
    x = np.arange(-image_half_width, image_half_width + dx, dx)
    xx, yy = np.meshgrid(x, x)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        z = np.sqrt(bb_radius**2 - xx**2 - yy**2) / bb_radius

    z[np.isnan(z)] = 0
    
    sig = profiles.scaled_penumbra_sig() * penumbra
    sig_pixel = sig / dx
    
    filtered = scipy.ndimage.gaussian_filter(z, sig_pixel)
    interp = scipy.interpolate.RegularGridInterpolator((x, x), filtered, bounds_error=False, fill_value=None)
    
    def bb_attenuation(x, y):        
        return 1 - bb_interp((x, y)) * max_attenuation
    
    return bb_attenuation

In [ ]:
dx = 0.1

bb_radius = bb_diameter / 2
bb_penumbra = penumbra_width / 3
image_half_width = bb_penumbra * 2 + bb_radius

x = np.arange(-image_half_width, image_half_width + dx, dx)
xx, yy = np.meshgrid(x, x)

z = np.sqrt(bb_radius**2 - xx**2 - yy**2) / bb_radius
z[np.isnan(z)] = 0

plt.pcolormesh(xx, yy, z)
plt.axis('equal')
plt.colorbar()

In [ ]:
sig = profiles.scaled_penumbra_sig() * bb_penumbra
sig_pixel = sig / dx

In [ ]:
filtered = scipy.ndimage.gaussian_filter(z, sig_pixel)
plt.pcolormesh(xx, yy, filtered)
plt.axis('equal')

In [ ]:
bb_interp = scipy.interpolate.RegularGridInterpolator((x, x), filtered, bounds_error=False, fill_value=None)

In [ ]:
result = bb_interp((xx, yy))
plt.pcolormesh(xx, yy, result)
plt.colorbar()
plt.axis('equal')

In [ ]:
bb_attenuation = 0.3
result = 1 - bb_interp((xx, yy)) * bb_attenuation

plt.pcolormesh(xx, yy, result)
plt.colorbar()
plt.axis('equal')

In [ ]:
bb_attenuation_map = create_bb_attenuation_func(bb_diameter, penumbra_width/3, bb_attenuation)

result = bb_attenuation_map(xx, yy)

plt.pcolormesh(xx, yy, result)
plt.colorbar()
plt.axis('equal')

In [ ]:
field_centre = [0, 0]
side_lengths = [20, 24]
penumbra_width = 2
rotation = 20

bb_centre = [2, 2]
bb_diameter = 8
bb_attenuation = 0.3


def create_field_with_bb_func(field_centre, bb_centre, side_lengths, bb_diameter, penumbra_width, rotation, bb_attenuation):
    field = profiles.create_rectangular_field_function(
        field_centre, side_lengths, penumbra_width, rotation)
    
    bb_penumbra = penumbra_width / 3
    bb_attenuation_map = create_bb_attenuation_func(bb_diameter, bb_penumbra, bb_attenuation)
    
    def field_with_bb(x, y):
        return field(x, y) * bb_attenuation_map(x - bb_centre[0], y - bb_centre[1])
    
    return field_with_bb


field_with_bb = create_field_with_bb_func(field_centre, bb_centre, side_lengths, bb_diameter, penumbra_width, rotation, bb_attenuation)

In [ ]:
show_field_with_profiles(field_with_bb)